In [ ]:
import os
os.chdir('C:\\Github')

In [ ]:
from ctypes import *
import random
from os import listdir
from os.path import isfile, isdir, join
import time
from queue import Queue
from collections import defaultdict
import csv
import pandas as pd
import numpy as np
import ast
import json
from PIL import Image
#import imagehash
import hashlib
import math
from statsmodels.tsa.api import ExponentialSmoothing, Holt, SimpleExpSmoothing

In [ ]:
table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

In [ ]:
scripted_walk = [
    'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
    'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
]

stationary = [
    'stationary_1'
]

freewalk = [
    'freewalk_1','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_8','freewalk_9'
]


walk_list = freewalk


methods_list =  [f'dev_1_A51',f'dev_1_U11',f'dev_1_U19e',f'dev_1_025',\
                 f'dev_2',f'dev_3',f'dev_4'
]

methods = 'dev_4'

beacon_count = 7  # 7 6 5 4 
beacon_rate = 1
# 1 GOOD 
# 2 1 Bad   025
# 3 2 Bad   025  U19e

In [ ]:
def signed_encode(hash_):
    # signed compute
    hash_ = np.where(hash_ > 0, hash_, 0)
    hash_ = np.where(hash_ <= 0, hash_, 1)
    
    return hash_.astype(int)

def text_hash(beacon_id, RSSI):
### Polynomial Enhancing ###
    RSSI_7 = 3*RSSI*RSSI*RSSI
    if walk_list == freewalk :
        RSSI_7 = 3*RSSI*RSSI + 2*RSSI*RSSI*RSSI + 1*RSSI*RSSI*RSSI*RSSI  #freewalk
    if walk_list == scripted_walk :
        RSSI_7 = 1*RSSI + 3*RSSI*RSSI*RSSI + 2*RSSI*RSSI*RSSI*RSSI       #scripted_walk
    weight = abs(RSSI_7*10)
### Polynomial Enhancing ###    

    text_ = hashlib.blake2b(beacon_id.encode()).hexdigest()
    text_encode = np.array([])
    
    for x in text_:
        value_ = int(x, base=16)
        tmp = '{0:04b}'.format(value_)
        value_list = [int(char) for char in tmp]
        text_encode = np.append(text_encode, value_list)
        text_encode = text_encode.astype(int)

    # weighted
    text_encode = np.where(text_encode < 0, text_encode, text_encode*weight)
    text_encode = np.where(text_encode > 0, text_encode, -1*weight)
    
    return text_encode

def similarity(hash1_, hash2_):
    return 1 - (((abs(hash1_- hash2_)).sum())/len(hash1_))

In [ ]:
#Wireless Train Hashing
name_type = 'beacon'
wireless_path = f'./data/0318_92589_train/'
Wireless_Train = pd.read_csv(join(wireless_path, 'wireless_training_set.csv'))

list_of_Wireless_Train_hash = []
train_label = []

# 將 train data 轉換成 hashing
for i in range(len(Wireless_Train)):
    Wireless_Train_row = Wireless_Train.iloc[i].to_dict()
    Wireless_Train_row_label = Wireless_Train_row['label']
    train_label.append(Wireless_Train_row_label)
    Wireless_Train_row.pop('label',None)
    if beacon_count == 6 :
        Wireless_Train_row.pop('Beacon_7',None)
    if beacon_count == 5 :
        Wireless_Train_row.pop('Beacon_3',None)
        Wireless_Train_row.pop('Beacon_4',None)
    if beacon_count == 4 :
        Wireless_Train_row.pop('Beacon_7',None)
        Wireless_Train_row.pop('Beacon_3',None)
        Wireless_Train_row.pop('Beacon_4',None)

    # device1 hash encode
    Wireless_Train_hash = np.array([])
    for beacon_id, RSSI in Wireless_Train_row.items():
        hash_ = text_hash(beacon_id, RSSI)
        if len(Wireless_Train_hash) < 1:
            Wireless_Train_hash = hash_
        else:
            Wireless_Train_hash = Wireless_Train_hash + hash_
           
    Wireless_Train_hash = signed_encode(Wireless_Train_hash)
    list_of_Wireless_Train_hash.append(Wireless_Train_hash)

In [ ]:
#Test Hashing 一起討論後推舉出候選人投票 寫給 image &pf
list_MDE = []
total_DE = 0
len_of_all_label = 0
K = 45

error_distri = [0]*16
error_distribution = []

for walk in walk_list:
    print("Walk Method : ",walk)
    hash_similarity = []

    test_label = []
    predict_label = []

    file_name = f'./wireless_to_pf/92589_wtp_{methods}_{walk}_bc_{beacon_count}_br_{beacon_rate}.csv'
    
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['label',methods])
        
        # 讀取 test data
        Wireless_Test = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/GalaxyA51/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        Wireless_Test_device2 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU11/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        Wireless_Test_device3 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU19e/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        Wireless_Test_device4 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/sharp025/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        
        #設定 wireless_reference_point 輸出
        wireless_reference_point_path = f'./wireless_reference_point/{methods}_{walk}_bc_{beacon_count}_br_{beacon_rate}_wireless_reference_point.csv'
        with open(wireless_reference_point_path, 'w', newline='') as csvfile_2 :
            wireless_reference_point = csv.writer(csvfile_2)  
            header = []
            header.append('label')
            header.append('wireless_reference_point')
            header.append('voter_wireless_reference_point')
            wireless_reference_point.writerow(header)
        
        # 將 test data 轉換成 hashing
            for j in range(len(Wireless_Test)):
        #Device1
                Wireless_Test_row = Wireless_Test.iloc[j].to_dict()
                Wireless_Test_row_label = Wireless_Test_row['label']
                test_label.append(Wireless_Test_row_label)
                Wireless_Test_row.pop('label',None)
                if beacon_count == 6 :
                    Wireless_Test_row.pop('Beacon_7',None)
                if beacon_count == 5 :
                    Wireless_Test_row.pop('Beacon_3',None)
                    Wireless_Test_row.pop('Beacon_4',None)
                if beacon_count == 4 :
                    Wireless_Test_row.pop('Beacon_7',None)
                    Wireless_Test_row.pop('Beacon_3',None)
                    Wireless_Test_row.pop('Beacon_4',None)

                # device1 hash encode
                Wireless_Test_hash = np.array([])            
                for beacon_id, RSSI in Wireless_Test_row.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    if len(Wireless_Test_hash) < 1:
                        Wireless_Test_hash = hash_
                    else:
                        Wireless_Test_hash = Wireless_Test_hash + hash_
        #Device1

        #Device2            
                Wireless_Test_row_device2 = Wireless_Test_device2.iloc[j].to_dict()
                Wireless_Test_row_device2.pop('label',None)
                if beacon_count == 6 :
                    Wireless_Test_row_device2.pop('Beacon_7',None)
                if beacon_count == 5 :
                    Wireless_Test_row_device2.pop('Beacon_3',None)
                    Wireless_Test_row_device2.pop('Beacon_4',None)
                if beacon_count == 4 :
                    Wireless_Test_row_device2.pop('Beacon_7',None)
                    Wireless_Test_row_device2.pop('Beacon_3',None)
                    Wireless_Test_row_device2.pop('Beacon_4',None)

                # device2 hash encode           
                for beacon_id, RSSI in Wireless_Test_row_device2.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    Wireless_Test_hash = Wireless_Test_hash + hash_ #hTCU11
        #Device2

        #Device3            
                Wireless_Test_row_device3 = Wireless_Test_device3.iloc[j].to_dict()
                Wireless_Test_row_device3.pop('label',None)
                if beacon_count == 6 :
                    Wireless_Test_row_device3.pop('Beacon_7',None)
                if beacon_count == 5 :
                    Wireless_Test_row_device3.pop('Beacon_3',None)
                    Wireless_Test_row_device3.pop('Beacon_4',None)
                if beacon_count == 4 :
                    Wireless_Test_row_device3.pop('Beacon_7',None)
                    Wireless_Test_row_device3.pop('Beacon_3',None)
                    Wireless_Test_row_device3.pop('Beacon_4',None)
                    
                # device3 hash encode           
                for beacon_id, RSSI in Wireless_Test_row_device3.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    Wireless_Test_hash = Wireless_Test_hash + hash_  #hTCU19e
        #Device3

        #Device4            
                Wireless_Test_row_device4 = Wireless_Test_device4.iloc[j].to_dict()
                Wireless_Test_row_device4.pop('label',None)
                if beacon_count == 6 :
                    Wireless_Test_row_device4.pop('Beacon_7',None)
                if beacon_count == 5 :
                    Wireless_Test_row_device4.pop('Beacon_3',None)
                    Wireless_Test_row_device4.pop('Beacon_4',None)
                if beacon_count == 4 :
                    Wireless_Test_row_device4.pop('Beacon_7',None)
                    Wireless_Test_row_device4.pop('Beacon_3',None)
                    Wireless_Test_row_device4.pop('Beacon_4',None)
                    
                # device4 hash encode           
                for beacon_id, RSSI in Wireless_Test_row_device4.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    Wireless_Test_hash = Wireless_Test_hash + hash_  #sharp025
        #Device4

                Wireless_Test_hash = signed_encode(Wireless_Test_hash)

                #找出 Top K 個像的 # 將 train data 與 test data 做比對
                k_top_similarity = [0.0]*K
                voter = [0.0]*K
                for k in range(len(list_of_Wireless_Train_hash)) :
                    sim_ = similarity(list_of_Wireless_Train_hash[k], Wireless_Test_hash)
                    if sim_ > (min(k_top_similarity)) :
                        k_top_similarity[k_top_similarity.index(min(k_top_similarity))] = sim_
                        voter[k_top_similarity.index(min(k_top_similarity))] = train_label[k]

                writer.writerow([Wireless_Test_row_label,voter])

                
                predict_label.append(max(voter, key = voter.count)) #票票等值
                print("Ground Truth :" , Wireless_Test_row_label)
                print("predict_label :",max(voter, key = voter.count))
            
                reference_point = []
                reference_point.append(Wireless_Test_row_label)
                reference_point.append(max(voter, key = voter.count))
                reference_point.append(voter)
                
                wireless_reference_point.writerow(reference_point)
    
    
#計算 MDE
    MDE = 0
    x = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,11,11,11,11,11,11,11,11,11,11, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11]
    y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,11,11,11,11,11,11,11,11,11,11]

    for i in range(len(test_label)):
        x_test_label = x[int(test_label[i])]
        y_test_label = y[int(test_label[i])]
        x_predict_label = x[int(predict_label[i])]
        y_predict_label = y[int(predict_label[i])]
        error = ((x_test_label - x_predict_label)**2 + (y_test_label - y_predict_label)**2)**0.5
        error_distri[math.ceil(error)] += 1
        MDE += error
        error_distribution.append(error*0.6)

    total_DE += MDE*0.6
    len_of_all_label += len(test_label)
    MDE = MDE*0.6/len(test_label)
    MDE = round(MDE, 4)
    list_MDE.append(MDE)

    print(methods,"in" ,walk," Mean Distance Error is : ", MDE)    
#計算 MDE      
    
print(list_MDE)
print(walk_list)
MDE = total_DE/len_of_all_label
print("Mean Distance Error of ",methods,"in " ,walk,"is : ",MDE)
print(error_distri)